In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2
from sklearn.utils import class_weight
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from glob import glob
from sklearn.metrics import confusion_matrix, roc_curve

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

In [ ]:
train_data_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train'
val_data_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation'

In [ ]:
IMAGE_SIZE = 224

# training config:
epochs = 10
batch_size = 32
normalization_factor = 255

#define paths
glaucoma_positive_train_path = '/content/drive/My Drive/Glaucoma_Detection/Dataset/Train/Glaucoma_Positive'
glaucoma_negative_train_path = '/content/drive/My Drive/Glaucoma_Detection/Dataset/Train/Glaucoma_Negative'

glaucoma_positive_test_path = '/content/drive/My Drive/Glaucoma_Detection/Dataset/Test/Glaucoma_Positive'
glaucoma_negative_test_path = '/content/drive/My Drive/Glaucoma_Detection/Dataset/Test/Glaucoma_Negative'

# Use glob to grab images from path .jpg or jpeg
glaucoma_positive_train_files = glob(glaucoma_positive_train_path + '/*')
glaucoma_negative_train_files = glob(glaucoma_negative_train_path + '/*')

glaucoma_positive_test_files = glob(glaucoma_positive_test_path + '/*')
glaucoma_negative_test_files = glob(glaucoma_negative_test_path + '/*')

In [ ]:
glaucoma_positive_train_files = glob(glaucoma_positive_train_path + '/*')
glaucoma_negative_train_files = glob(glaucoma_negative_train_path + '/*')
glaucoma_positive_test_files = glob(glaucoma_positive_test_path + '/*')
glaucoma_negative_test_files = glob(glaucoma_negative_test_path + '/*')

glaucoma_positive_train_images = []
glaucoma_negative_train_images = []
glaucoma_positive_test_images = []
glaucoma_negative_test_images = []

for file in glaucoma_positive_train_files:
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMAGE_SIZE, IMAGE_SIZE))
    glaucoma_positive_train_images.append(img)

for file in glaucoma_negative_train_files:
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMAGE_SIZE, IMAGE_SIZE))
    glaucoma_negative_train_images.append(img)

for file in glaucoma_positive_test_files:
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMAGE_SIZE, IMAGE_SIZE))
    glaucoma_positive_test_images.append(img)

for file in glaucoma_negative_test_files:
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(IMAGE_SIZE, IMAGE_SIZE))
    glaucoma_negative_test_images.append(img)

In [ ]:
import os

# Verify that the directories are correct and the images are present
train_glaucoma_positive_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Positive'
train_glaucoma_negative_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Negative'
test_glaucoma_positive_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Positive'
test_glaucoma_negative_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative'

# List the images in each directory
glaucoma_positive_train_images = os.listdir(train_glaucoma_positive_dir)
glaucoma_negative_train_images = os.listdir(train_glaucoma_negative_dir)
glaucoma_positive_test_images = os.listdir(test_glaucoma_positive_dir)
glaucoma_negative_test_images = os.listdir(test_glaucoma_negative_dir)

# Print the number of images in each folder
print(f"Glaucoma Positive Train Images: {len(glaucoma_positive_train_images)}")
print(f"Glaucoma Negative Train Images: {len(glaucoma_negative_train_images)}")
print(f"Glaucoma Positive Test Images: {len(glaucoma_positive_test_images)}")
print(f"Glaucoma Negative Test Images: {len(glaucoma_negative_test_images)}")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

def plot_images(image_filenames, image_dir, title, max_images=20):
    images = []

    for fname in image_filenames[:max_images]:
        img_path = os.path.join(image_dir, fname)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        images.append(img_array)

    n = len(images)
    nrows, ncols = (n // 4 + 1, 4)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 8))
    ax = ax.flatten()

    for i in range(n):
        ax[i].imshow(images[i])
        ax[i].axis('off')

    plt.suptitle(title, fontsize=20)
    plt.tight_layout()
    plt.show()
train_pos_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Positive'
train_neg_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Negative'
test_pos_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Positive'
test_neg_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative'

# Plot
plot_images(glaucoma_positive_train_images, train_pos_dir, 'Glaucoma Positive Train')
plot_images(glaucoma_negative_train_images, train_neg_dir, 'Glaucoma Negative Train')
plot_images(glaucoma_positive_test_images, test_pos_dir, 'Glaucoma Positive Test')
plot_images(glaucoma_negative_test_images, test_neg_dir, 'Glaucoma Negative Test')


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_images(image_dir, image_filenames, target_size=(224, 224)):
    images = []
    for fname in image_filenames:
        img_path = os.path.join(image_dir, fname)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img) / 255.0  # Normalization to [0, 1]
        images.append(img_array)
    return np.array(images)

train_pos_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Positive'
train_neg_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Train/Glaucoma_Negative'
test_pos_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Positive'
test_neg_dir = '/content/drive/My Drive/Fundus_Scanes_Sorted/Validation/Glaucoma_Negative'

# Load and normalize images
glaucoma_positive_train_images = load_and_preprocess_images(train_pos_dir, glaucoma_positive_train_images)
glaucoma_negative_train_images = load_and_preprocess_images(train_neg_dir, glaucoma_negative_train_images)
glaucoma_positive_test_images = load_and_preprocess_images(test_pos_dir, glaucoma_positive_test_images)
glaucoma_negative_test_images = load_and_preprocess_images(test_neg_dir, glaucoma_negative_test_images)

In [ ]:
glaucoma_negative_train_labels = [0] * len(glaucoma_negative_train_images)
glaucoma_positive_train_labels = [1] * len(glaucoma_positive_train_images)

glaucoma_negative_test_labels = [0] * len(glaucoma_negative_test_images)
glaucoma_positive_test_labels = [1] * len(glaucoma_positive_test_images)

# Combine images
X_train = np.concatenate((glaucoma_negative_train_images, glaucoma_positive_train_images), axis=0)
X_test = np.concatenate((glaucoma_negative_test_images, glaucoma_positive_test_images), axis=0)

# Combine labels
y_train = np.concatenate((glaucoma_negative_train_labels, glaucoma_positive_train_labels), axis=0)
y_test = np.concatenate((glaucoma_negative_test_labels, glaucoma_positive_test_labels), axis=0)

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [ ]:
# Convert one-hot labels back to class indices (0 or 1)
y_train_labels = np.argmax(y_train, axis=1)

# Compute weights: this gives higher importance to minority class
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

# Format as a dictionary
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weights_dict)


In [ ]:
import matplotlib.pyplot as plt

def plot_images(images, title, max_images=20):
    n_images = min(len(images), max_images)
    nrows, ncols = 4, 5  # 4x5 = 20 images
    figsize = [12, 8]

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, facecolor=(1, 1, 1))

    for i, axi in enumerate(ax.flat):
        if i < n_images:
            axi.imshow(images[i])
        else:
            axi.axis('off')
        axi.set_axis_off()

    plt.suptitle(title, fontsize=20)
    plt.tight_layout(pad=0.5, rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)

# Replace Flatten with GlobalAveragePooling2D
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze more than 50 layers to better learn retinal features
for layer in base_model.layers[-100:]:
    layer.trainable = True
 # Freeze all layers initially

from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']  # Add AUC metric
)
model.summary()

In [ ]:
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,  # Critical for fundus images
    fill_mode='nearest'
)

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6)

history_fine = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=20,
    callbacks=[early_stop, reduce_lr],
    class_weight=class_weights_dict,  # <-- 👈 KEY FIX
    verbose=1
)

In [ ]:
# Unfreeze top 50 layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history_fine = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=20,  # Fine-tuning epochs
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

y_pred = model.predict(X_test, batch_size=batch_size)

# Prediction visualization (first 10 images)
for i in range(10):
    plt.imshow(X_test[i])
    prob = y_pred[i][1]
    if prob > 0.5:
        plt.title(f"{prob*100:.1f}% Glaucoma Positive")
    else:
        plt.title(f"{(1-prob)*100:.1f}% Glaucoma Negative")
    plt.show()

# Confusion Matrix
y_pred_bin = np.argmax(y_pred, axis=1)
y_test_bin = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_bin, y_pred_bin)
sns.heatmap(cm, annot=True, fmt='d', cmap='plasma')
plt.xticks([0.5,1.5], ['Glaucoma Negative','Glaucoma Positive'])
plt.yticks([0.5,1.5], ['Glaucoma Negative','Glaucoma Positive'])
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test_bin, y_pred[:,1])
plt.plot(fpr, tpr)
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)
plt.show()

# Classification Report
print(classification_report(y_test_bin, y_pred_bin,
                           target_names=['Glaucoma Negative', 'Glaucoma Positive']))

In [ ]:
y_pred_bin = np.argmax(y_pred, axis=1)
y_test_bin = np.argmax(y_test, axis=1)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_bin, y_pred_bin)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 12
plt.title('ROC curve for our model')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)

In [ ]:
def plot_confusion_matrix(normalize):
  classes = ['Glaucoma Positive','Glaucoma Negative']
  tick_marks = [0.5,1.5]
  cn = confusion_matrix(y_test_bin, y_pred_bin,normalize=normalize)
  sns.heatmap(cn,cmap='plasma',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Confusion Matrix')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize=None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize='true')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('inception_resnet_v2_glaucoma_accuracy.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])
plt.savefig('inception_resnet_v2_glaucoma_loss.png')
plt.show()